<a href="https://colab.research.google.com/github/Sergjey/AI-LLM/blob/main/AI_Smart_Home_Speaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#prepare env
!pip install --upgrade langchain
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.1/816.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00


In [ ]:
#Let's upload voice file with song request to the Colab server:
from google.colab import files
upload = files.upload()

Saving song_request_example.m4a to song_request_example.m4a


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
#setting instructions
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
response_schemas = [
    ResponseSchema(name="artist", description="The name of the musical artist"),
    ResponseSchema(name="song", description="The name of the song that the artist plays")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```


In [ ]:
#prompt creating
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
song_prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("Given a command from the user, extract the artist and song names \n \
                                                    {format_instructions}\n{user_prompt}")
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
# Creating the smart speaker
import langchain
from langchain.agents import tool, load_tools, initialize_agent, AgentType
from datetime import date
from langchain.chat_models import ChatOpenAI

@tool
def song_player(user_message:str)->str:
    """
    Use this function whenever the user's message contains one of the words:
    turn on
    play
    I want to listen to
    """
    query = song_prompt.format_prompt(user_prompt=user_message)
    chat_model = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')
    output = chat_model(query.to_messages())
    output = output_parser.parse(output.content)
    if output['artist'] != '':
        #TODO search for the song in the library
        #TODO play the song
        print('')
        print('Song title extracted from raw text:', output)
        print('Searching for the song in the library... Song found')
        print(f'Playing the song {output["song"]} by {output["artist"]}')
        return 'Song played'
    else:
        print('Song not found')
        return 'Song not found'

@tool
def leave_house1(txt):
    """
    Use this function whenever the user's message contains one of the words:
    I left the house
    leaving the house
    we left the house
    I left home
    """
    #TODO send commands to smart home devices via API
    print('')
    print('Smart home control activated')
    print('Turning off all sockets except for the fridge')
    print('Closing water taps')
    return True

@tool
def leave_house(txt):
    """
    If the user has left the house - run this function
    """
    #TODO send commands to smart home devices via API
    print('')
    print('Smart home control activated')
    print('Turning off all sockets except for the fridge')
    print('Closing water taps')
    return True

@tool
def temperature_in_room(val:str)->str:
    """
    This function allows understanding whether it's cold or not in the room
    """
    #TODO request the temperature from the thermometer in the room
    room_temp = 21
    #TODO get the minimum comfortable temperature from the settings
    min_temp = 25
    if room_temp < min_temp:
        message = 'it is cold in the room'
    else:
        message = 'it is comfortable in the room'
    return message

llm = ChatOpenAI(temperature=0,
              model_name="gpt-3.5-turbo",
             )
tools = load_tools([], llm=llm)
tools.append(song_player)
tools.append(leave_house)
tools.append(temperature_in_room)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors = True,
    verbose=True)

#langchain.debug = True
langchain.debug = False


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
import openai
from openai.types.audio import Transcription

# Setting API key from OpenAI
openai.api_key = ""

# Open the audio file for reading in binary mode.
audio_file = open("song_request_example.m4a", "rb")

# Creating a transcription of the audio file
transcript = openai.audio.transcriptions.create(
  model="whisper-1",
  file=audio_file,
  response_format="text"
)

# Display the transcription.
print(transcript)

agent.run(transcript)

I wanna listen to Imagine Dragons Believer.



> Entering new AgentExecutor chain...
The user wants to listen to a specific song by Imagine Dragons.
Action: song_player
Action Input: user_message = "I wanna listen to Imagine Dragons Believer"

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Song title extracted from raw text: {'artist': 'Imagine Dragons', 'song': 'Believer'}
Searching for the song in the library... Song found
Playing the song Believer by Imagine Dragons

Observation: Song played
Thought:The user's request has been fulfilled.
Final Answer: The song Imagine Dragons Believer has been played.

> Finished chain.


'The song Imagine Dragons Believer has been played.'